### Connect to Neo4j browser

In [1]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
import random
import warnings
warnings.filterwarnings('ignore')

# Connect to Neo4j
neo = pd.read_csv('Neo4j_pass.csv')
uri, auth1, auth2 = neo.at[0,'neo4j'], neo.at[1,'neo4j'], neo.at[2,'neo4j']

with GraphDatabase.driver(uri, auth = (auth1,auth2)) as driver:
    driver.verify_connectivity()
    
session = driver.session()

# Delete existing data
session.run('match (n) detach delete n')

### Generate sample data

In [2]:
random.seed(42)
data = {'sender':[], 'recipient':[], 'amount_usd':[], 'num_trxs':[], 'action':[]}

n_nodes = 10
n_loops = 14

for _ in range(n_loops):
    sender     = 'Wallet_{}'.format(random.randint(1,n_nodes))
    recipient  = 'Wallet_{}'.format(random.randint(1,n_nodes))
    amount_usd = random.randint(1, 100000)
    num_trxs   = random.randint(1, 100)
    action     = random.choice(['Payment','Payout','VA']) 

    data['sender'].append(sender)
    data['recipient'].append(recipient)
    data['amount_usd'].append(amount_usd)
    data['num_trxs'].append(num_trxs)
    data['action'].append(action)

df = pd.DataFrame(data)
df = df[df.sender != df.recipient].groupby(['sender','recipient','action'], as_index = False).\
                                   agg({'amount_usd':'sum','num_trxs':'sum'})

unique_values = np.union1d(df['sender'], df['recipient'])
sub_wallets = np.random.choice(unique_values, size = int(np.floor(len(unique_values)/2)), replace = False)

df['s_type'] = np.where(df.sender.isin(sub_wallets),'SubWallet','Wallet')
df['r_type'] = np.where(df.recipient.isin(sub_wallets),'SubWallet','Wallet')

df

,sender,recipient,action,amount_usd,num_trxs,s_type,r_type
0,Wallet_1,Wallet_2,VA,28658,30,SubWallet,SubWallet
1,Wallet_10,Wallet_1,VA,73564,26,Wallet,SubWallet
2,Wallet_10,Wallet_4,Payment,92350,9,Wallet,Wallet
3,Wallet_2,Wallet_1,Payment,97197,36,SubWallet,SubWallet
4,Wallet_4,Wallet_3,VA,96531,14,Wallet,SubWallet
5,Wallet_4,Wallet_5,Payment,10459,30,Wallet,SubWallet
6,Wallet_5,Wallet_1,VA,99459,21,SubWallet,SubWallet
7,Wallet_6,Wallet_2,Payment,12157,49,Wallet,SubWallet
8,Wallet_7,Wallet_6,Payment,36422,20,SubWallet,Wallet
9,Wallet_8,Wallet_9,Payment,16362,49,Wallet,Wallet


### Unique nodes

In [3]:
senders = df.loc[:,['sender','s_type']].drop_duplicates(subset = 'sender')
senders.columns = ['node','type']

recipient = df.loc[:,['recipient','r_type']].drop_duplicates(subset = 'recipient')
recipient.columns = ['node','type']

node_types = pd.concat([senders,recipient], ignore_index = True)
node_types.drop_duplicates(subset = 'node', ignore_index = True, inplace = True)

s1 = df.groupby('sender', as_index = False).agg(sent_usd = ('amount_usd','sum')).rename(columns = {'sender':'node'})
r1 = df.groupby('recipient', as_index = False).agg(received_usd = ('amount_usd','sum')).rename(columns = {'recipient':'node'})

node_types = node_types.merge(s1, on = 'node', how = 'left')
node_types = node_types.merge(r1, on = 'node', how = 'left')

for col in ['sent_usd','received_usd']:
    node_types[col] = node_types[col].fillna(0).astype(int)
    
# no. of connections
temp1 = df[['sender','recipient']].apply(lambda x: sorted(x), axis = 1, result_type = 'expand').drop_duplicates()
temp2 = pd.concat([temp1.iloc[:,0], temp1.iloc[:,1]], ignore_index = True)
temp3 = temp2.value_counts().reset_index().rename(columns = {'index':'node','count':'connections'})

node_types = node_types.merge(temp3, on = 'node', how = 'left')

node_types

,node,type,sent_usd,received_usd,connections
0,Wallet_1,SubWallet,28658,270220,3
1,Wallet_10,Wallet,165914,0,2
2,Wallet_2,SubWallet,97197,118213,3
3,Wallet_4,Wallet,106990,92350,3
4,Wallet_5,SubWallet,99459,92857,3
5,Wallet_6,Wallet,12157,36422,2
6,Wallet_7,SubWallet,36422,28894,2
7,Wallet_8,Wallet,16362,0,1
8,Wallet_9,Wallet,188690,16362,4
9,Wallet_3,SubWallet,0,96531,1


### Upload data

In [4]:
# Nodes
txt1 = []
for _, row in node_types.iterrows():
    node,node_type,s_usd,r_usd,conn = row['node'],row['type'],row['sent_usd'],row['received_usd'],row['connections']
    t1 = f"""MERGE ({node}:{node_type} {{connections: {conn}, name: "{node}", sent_usd: {s_usd}, received_usd: {r_usd}}})"""
    txt1.append(t1)
    
nodes_text = '\n'.join(txt1)

# Connections
txt2 = []
for _, row in df.iterrows():
    s,r,a,u,t = row['sender'],row['recipient'],row['action'],row['amount_usd'],row['num_trxs']
    t2 = f"""MERGE ({s})-[:{a}_${u} {{action: "{a}", amount_usd: {u}, transactions: {t}}}]->({r})"""
    txt2.append(t2)
    
conn_text = '\n'.join(txt2) 

final_text = f"{nodes_text}\n\n{conn_text}"

session.run(final_text)

https://browser.graphapp.io/

#### Cypher - simple select *
match (n) return n

#### Cypher - select with conditions
MATCH (n {name: 'Wallet_2'})--(related)
RETURN n, related